# Dependencies

In [941]:
#%pip install pyautogui opencv-python easyocr

In [942]:
#%pip install torch==2.3.1 torchaudio==2.3.1 torchvision==0.18.1

# Libraries

In [944]:
import os
import cv2
import easyocr
import logging
import pyautogui
import numpy as np
from collections import deque
import matplotlib.pyplot as plt

In [945]:
def take_screenshot():
    screenshot = pyautogui.screenshot()
    screenshot = np.array(screenshot)
    return cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)

def get_colors():
    return np.array([
        [81, 215, 170],  # Green light
        [73, 209, 162],  # Green
        [159, 194, 229], # Nude light
        [153, 184, 215], # Nude
        [58, 175, 135]   # line
    ], dtype=np.uint8)

def create_mask_colors(screenshot, colors):
    return [cv2.inRange(screenshot, color, color) for color in colors]

def get_table_contours(masks):
    combined_mask = np.bitwise_or.reduce(masks)
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def get_table_dimensions(contours):
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    return x, y, w, h

In [946]:
screenshot = take_screenshot()
colors = get_colors()
masks = create_mask_colors(screenshot, colors)

table_contours = get_table_contours(masks)
table_dims = get_table_dimensions(table_contours)

square_dims = {
    (450, 360): 45, # Easy
    (540, 420): 30, # Medium
    (600, 500): 25, # Hard
}

square_size = square_dims[(table_dims[2], table_dims[3])]

In [947]:
logging.getLogger().setLevel(logging.ERROR)
reader = easyocr.Reader(['en'])

In [948]:
def get_square(screenshot, table_dims, i, j, square_size):
    dx, dy = j * square_size, i * square_size
    x, y, w, h = (
        table_dims[0] + dx, 
        table_dims[1] + dy, 
        square_size, 
        square_size
    )
    return screenshot[y:y+h, x:x+w]

def scale_image(square, width, inter=cv2.INTER_AREA):
    original_height, original_width = square.shape[:2]
    if original_width == width:
        return square
    
    ratio = width / float(original_width)
    height = int(original_height * ratio)
    
    return cv2.resize(square, (width, height), interpolation=inter)

def str_to_number(str_value):
    try:
        return int(str_value)
    except ValueError:
        return None

def get_number(square):
    results = reader.readtext(square)
    if not results:
        return None

    sorted_results = sorted(results, key=lambda x: x[2], reverse=True)
    return int(sorted_results[0][1])
    
def get_cell_value(square, colors):
    for index, color in enumerate(colors):
        mask = cv2.inRange(square, color, color)
        color_percentage = (np.sum(mask) / (mask.size * 255)) * 100
        
        if color_percentage > 90:
            return 0 if index < 2 else -1
    
    return get_number(square)

In [949]:
_, _, width, height = table_dims
table = np.zeros((height // square_size, width // square_size))

for i in range(len(table)):
    for j in range(len(table[i])):
        square = get_square(screenshot, table_dims, i, j, square_size)
        resized_square = scale_image(square, 5 * square_size)
        table[i, j] = get_cell_value(resized_square, colors[:4])
    print(f"Fila {i + 1} de {len(table)} leida")

Fila 1 de 20 leida
Fila 2 de 20 leida
Fila 3 de 20 leida
Fila 4 de 20 leida
Fila 5 de 20 leida
Fila 6 de 20 leida
Fila 7 de 20 leida
Fila 8 de 20 leida
Fila 9 de 20 leida
Fila 10 de 20 leida
Fila 11 de 20 leida
Fila 12 de 20 leida
Fila 13 de 20 leida
Fila 14 de 20 leida
Fila 15 de 20 leida
Fila 16 de 20 leida
Fila 17 de 20 leida
Fila 18 de 20 leida
Fila 19 de 20 leida
Fila 20 de 20 leida


In [950]:
def show_table(table):
    str_value = ""
    for row in table:
        for col in row:
            value = f" {float(col):.2f}"
            str_value += value + ""
        str_value += "\n"

    print(str_value)

In [951]:
show_table(table)

 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 2.00 1.00 2.00 2.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 4.00 2.00 1.00 -1.00 -1.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 -1.00 -1.00 -1.00 1.00 2.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
 0.00 0.00 0.00 0.00 0.00 0

In [952]:
def get_cell(table, i, j):
    if 0 <= i < len(table) and 0 <= j < len(table[0]):
        return table[i][j]
    return -3

def get_adjacent_cells(table, i, j):
    adjacent_cells = []
    
    for dx, dy in [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]:
        ni, nj = i + dy, j + dx
        cell_value = get_cell(table, ni, nj)
        
        if cell_value != -3:
            adjacent_cells.append((ni, nj, cell_value))
    
    return adjacent_cells

def adjacent_cell_with_positive_numbers(table, i, j):
    return [(ni, nj, cell_value) for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value > 0]

def num_adjacent_cell_with_positive_numbers(table, i, j):
    return len(adjacent_cell_with_positive_numbers(table, i, j))

def mines_around_cell(table, i, j):
    return sum(1 for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value == -2)

def empty_cells_around(table, i, j):
    return {(ni, nj) for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value == 0}

def heuristic(table, i, j):
    mines_around, empty_cells = 0, set()
    adjacent_cells = adjacent_cell_with_positive_numbers(table, i, j)
    
    if not adjacent_cells or get_cell(table, i, j) != 0:
        return 0

    for ni, nj, cell_value in adjacent_cells:
        mines_around += cell_value - mines_around_cell(table, ni, nj)
        empty_cells.update(empty_cells_around(table, ni, nj))
        
    return mines_around / len(empty_cells) if empty_cells else int(mines_around > 0)

def special_case(table, i, j):
    cases = [
        [[(0, -2), (1, -2), (2, -2), (2, -1), (2, 0)], [(0, -1), (1, 0)], (1, -1)], # Arriba-Derecha
        [[(2, 0), (2, 1), (2, 2), (1, 2), (0, 2)], [(1, 0), (0, 1)], (1, 1)], # Derecha-Abajo
        [[(2, 0), (2, -1), (2, -2), (1, -2), (0, -2)], [(1, 0), (0, -1)], (1, -1)], # Abajo-Izquierda
        [[(-2, 0), (-2, -1), (-2, -2), (-1, -2), (0, -2)], [(-1, 0), (0, -1)], (-1, -1)], # Izquierda-Arriba
    ]

    if get_cell(table, i, j) == 0:
        for case in cases:
            is_lockeds = all(get_cell(table, i + dy, j + dx) < 0 for dx, dy in case[0])
            is_numbers = all(get_cell(table, i + dy, j + dx) > 0 for dx, dy in case[1])
            is_one = case[2] == 1

            if is_lockeds and is_numbers and is_one:
                return 100

    return heuristic(table, i, j)

In [953]:
class TreeNode:
    def __init__(self, table):
        self.table = table
        self.children = []

    def find_paths(self):
        def dfs(node, leaves):
            if node is None:
                return

            # Si el nodo actual es una hoja, agregarlo a la lista de hojas
            if not node.children:
                leaves.append(node.table)

            # Recorrer todos los hijos del nodo
            for child in node.children:
                dfs(child, leaves)

        leaves = []
        dfs(self, leaves)
        return leaves

In [954]:
def calculate_heuristics(table):
    heuristics = []
    for i in range(len(table)):
        for j in range(len(table[0])):
            value = special_case(table, i, j)
            if value > 0:
                heuristics.append((i, j, value))
    return heuristics

def is_a_valid_land_mine(table):
    for i in range(len(table)):
        for j in range(len(table[i])):
            cell_value = get_cell(table, i, j)
            if 0 < cell_value < mines_around_cell(table, i, j):
                return False
    return True

def is_possible_to_continue(table):
    for i in range(len(table)):
        for j in range(len(table[i])):
            cell = get_cell(table, i, j)
            if 0 < cell and mines_around_cell(table, i, j) < cell:
                return True
    return False

def calculate_probability(matrices, target_number, row_index, col_index):
    total_matrices = len(matrices)
    count = sum(matrix[row_index][col_index] == target_number for matrix in matrices)
    return 0 if total_matrices == 0 else (count / total_matrices), count > 0

def get_probabilities(table, paths, n, m):
    prob_table = np.ones((n, m))
    
    for i in range(n):
        for j in range(m):
            prob, condition = calculate_probability(paths, -2, i, j)
            num_adj = num_adjacent_cell_with_positive_numbers(table, i, j)
            cell = get_cell(table, i, j)
            
            if cell == -2 or cell == 0 and (condition or num_adj > 0):
                prob_table[i, j] = prob
            else:
                prob_table[i, j] = 1
                
    return prob_table

def find_min_position(matrix):
    np_matrix = np.array(matrix)
    return np.unravel_index(np.argmin(np_matrix), np_matrix.shape)

def find_best_movement(table, root, n, m):
    paths = root.find_paths()
    valid_matrices = [path for path in paths if not is_possible_to_continue(path)]
    prob_table = get_probabilities(table, valid_matrices, n, m)
    show_table(prob_table)
    return find_min_position(prob_table)

def table_to_tuple(table):
    return tuple(tuple(row) for row in table)

def generate_possibilities(root, visited_tables=set()):
    heuristics_list = calculate_heuristics(root.table)
    ordered_heuristics_list = sorted(heuristics_list, key=lambda x: x[2])[::-1]
    
    if not is_possible_to_continue(root.table) or not ordered_heuristics_list:
        return

    for land_mine in ordered_heuristics_list:
        i, j, _ = land_mine
        new_table = [row.copy() for row in root.table]
        new_table[i][j] = -2
        
        if is_a_valid_land_mine(new_table):
            table_tuple = table_to_tuple(new_table)
            
            if table_tuple not in visited_tables:
                visited_tables.add(table_tuple)
                node = TreeNode(new_table)

                root.children.append(node)
                generate_possibilities(node, visited_tables)

In [955]:
heuristics_list = calculate_heuristics(table)
ordered_heuristics_list = sorted(heuristics_list, key=lambda x: x[2])

In [956]:
i, j, _ = ordered_heuristics_list.pop()
new_table = [list(row).copy() for row in table]
new_table[i][j] = -2

In [ ]:
rootNode = TreeNode(new_table)
generate_possibilities(rootNode)

In [ ]:
square_i, square_j = find_best_movement(table, rootNode, len(table), len(table[0]))

In [ ]:
def show_best_movement(table_dims, i, j, square_size):
    x, y, w, h = table_dims
    sx, sy = j * square_size, i * square_size
        
    table = screenshot[y:y+h, x:x+w]    
    cv2.rectangle(table, (sx, sy), (sx+square_size, sy+square_size), (255, 0, 0), 2)

    plt.imshow(cv2.cvtColor(table, cv2.COLOR_BGR2RGB), cmap='gray')
    plt.axis('off')  # Ocultar los ejes
    plt.show()

In [ ]:
show_best_movement(table_dims, square_i, square_j, square_size)